In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import matplotlib as mpl
import cv2
import PIL
import time
import glob
import os
import imageio

In [2]:
(train_x, _), (_, _) = tf.keras.datasets.mnist.load_data()# train말고 필없음
train_x = (train_x - 127.5) / 127.5  # 표준화
X_train = (
    tf.data.Dataset.from_tensor_slices(train_x)
    .shuffle(60000) #섞어섞어
    .batch(512, drop_remainder=True) #512개씩 나누고 남은 거 제거
    .repeat()
)

11501568/11490434 [==============================] - 0s 0us/step


In [3]:
# metrics 지정
g_loss_metrics = tf.metrics.Mean(name='g_loss')
d_loss_metrics = tf.metrics.Mean(name='d_loss')
total_loss_metrics = tf.metrics.Mean(name='total_loss')

In [4]:
# 생성자에 쓸 노이즈
def make_noise(input_shape):
  return tf.random.normal(input_shape)

In [5]:
IMAGE_SHAPE=(28,28,1)

In [6]:
# 손실함수 두개
def get_loss():
  def d_loss(real_logits,fake_logits):
    return -tf.reduce_mean(tf.math.log(real_logits+(1e-10)+tf.math.log(1.-fake_logits+1e-10)))
  def g_loss(fake_logits):
    return -tf.reduce_mean(tf.math.log(fake_logits+1e-10))
  return d_loss,g_loss

In [7]:
def make_generator(input_shape):
  return tf.keras.Sequential([
                              layers.Dense(256,activation="relu",input_shape=input_shape),
                              layers.Dense(256,activation="relu"),
                              layers.Dense(784,activation="sigmoid"),
                              layers.Reshape(IMAGE_SHAPE)
  ])
def make_discriminator(input_shape):
  return tf.keras.Sequential([
                              layers.Input(IMAGE_SHAPE),
                              layers.Flatten(),
                              layers.Dense(256,input_shape=input_shape),
                              layers.Dropout(rate=0.3),
                              layers.Dense(256),
                              layers.Dropout(rate=0.3),
                              layers.Dense(1,activation="sigmoid")
  ])

In [8]:
Z_DIM=100
# G,D생성
G = make_generator((Z_DIM,))
D = make_discriminator((IMAGE_SHAPE))

In [9]:
# 옵티마이저 생성
g_optim = tf.keras.optimizers.Adam(0.0004)
d_optim = tf.keras.optimizers.Adam(0.0004)
d_loss,g_loss=get_loss()

In [10]:
def get_random_z(z_dim, batch_size):
    return tf.random.uniform([batch_size, z_dim], minval=-1, maxval=1)

In [19]:
def train_step(real_images):
  z=get_random_z(Z_DIM,512)
  with tf.GradientTape() as d_tape,tf.GradientTape() as g_tape:
    fake_images=G(z,training=True) # G로 이미지 생성
    fake_logits=D(fake_images,training=True) # 생성 이미지 판별
    real_logits=D(real_images,training=True) # 실제 이미지 판별
    global d_loss,g_loss
    d_loss = d_loss(real_logits, fake_logits)
    g_loss = g_loss(fake_logits)
  d_gradients=d_tape.gradient(d_loss,D.trainable_variables)
  g_gradients=g_tape.gradient(g_loss,G.trainable_variables)

  d_optim.apply_gradients(zip(d_gradients, D.trainable_variables))
  g_optim.apply_gradients(zip(g_gradients, G.trainable_variables))

  return g_loss, d_loss

In [20]:
get_random_z(Z_DIM,512).shape

TensorShape([512, 100])

In [23]:
def train(ds, log_freq=20):
    ds = iter(ds)
    for step in range(50):
        images = next(ds)
        g_loss, d_loss = train_step(images)

        g_loss_metrics(g_loss)
        d_loss_metrics(d_loss)
        total_loss_metrics(g_loss + d_loss)

        if step % log_freq == 0:
            template = '[{}/{}] D_loss={:.5f} G_loss={:.5f} Total_loss={:.5f}'
            print(template.format(step, 50, d_loss_metrics.result(),
                                  g_loss_metrics.result(), total_loss_metrics.result()))
            g_loss_metrics.reset_states()
            d_loss_metrics.reset_states()
            total_loss_metrics.reset_states()

In [25]:
train(X_train)

TypeError: ignored

In [ ]:
noise = tf.random.normal([1, 100])
zeros=np.zeros([1,100])

In [ ]:
plt.imshow(G(noise,training=False)[0,:,:])